In [1]:
%pip install python-chess

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import chess

def evaluate_board(board):
    piece_values = {
        chess.PAWN: 1,
        chess.KNIGHT: 3,
        chess.BISHOP: 3,
        chess.ROOK: 5,
        chess.QUEEN: 9,
        chess.KING: 0
    }

    # Adjust piece values based on remaining pieces
    total_pieces = len(board.piece_map())
    if total_pieces <= 16:
        piece_values[chess.BISHOP] = 4
    if total_pieces > 16:
        piece_values[chess.KNIGHT] = 3
    else:
        piece_values[chess.KNIGHT] = 4

    # Calculate material
    white_material = sum(piece_values.get(piece.piece_type, 0)
                        for square, piece in board.piece_map().items()
                        if piece.color == chess.WHITE)
    black_material = sum(piece_values.get(piece.piece_type, 0)
                        for square, piece in board.piece_map().items()
                        if piece.color == chess.BLACK)
    material_score = white_material - black_material if board.turn == chess.WHITE else black_material - white_material

    positional_score = 0

    # Center control
    center_squares = [chess.D4, chess.D5, chess.E4, chess.E5]
    sub_center_squares = [chess.C6, chess.C5, chess.C4, chess.C3, chess.D6, chess.D3, chess.E3, chess.E6, chess.F6, chess.F5, chess.F4, chess.F3]

    for square, piece in board.piece_map().items():
        # Center control bonuses
        if square in center_squares:
            positional_score += 0.1 if piece.color == board.turn else -0.05
        if square in sub_center_squares:
            positional_score += 0.075 if piece.color == board.turn else -0.05

        # Protection bonuses
        if is_protected(board, square):
            positional_score += 0.05

        # Pawn structure
        if piece.piece_type == chess.PAWN:
            if is_pawn_protected(board, square):
                positional_score += 0.1
            positional_score += evaluate_pawn_structure(board, square)

        # King safety
        if piece.piece_type == chess.KING:
            positional_score += evaluate_king_safety(board, square)

    # Double pawn penalty
    for file in range(8):
        pawns_on_file = sum(1 for square, piece in board.piece_map().items()
                           if piece.piece_type == chess.PAWN
                           and chess.square_file(square) == file
                           and piece.color == board.turn)
        if pawns_on_file > 1:
            positional_score -= (pawns_on_file - 1) * 0.5

    # Mobility evaluation
    mobility = len(list(board.legal_moves))
    positional_score += mobility * 0.01

    # Check for blunders, pins, and forks
    positional_score += evaluate_tactics(board, piece_values)

    total_score = material_score + positional_score

    # Checkmate detection bonus
    if board.is_checkmate():
        total_score += 100 if board.turn != board.turn else -100

    return total_score

def is_protected(board, square):
    """Check if a piece is protected by another piece"""
    piece = board.piece_at(square)
    if not piece:
        return False

    for move in board.legal_moves:
        if move.to_square == square and board.piece_at(move.from_square).color == piece.color:
            return True
    return False

def is_pawn_protected(board, square):
    """Check if a pawn is protected by another pawn"""
    piece = board.piece_at(square)
    if not piece or piece.piece_type != chess.PAWN:
        return False

    file = chess.square_file(square)
    rank = chess.square_rank(square)
    pawn_color = piece.color

    # Check diagonally adjacent squares for protecting pawns
    for dx in [-1, 1]:
        if 0 <= file + dx < 8:
            if pawn_color == chess.WHITE and rank > 0:
                protector = chess.square(file + dx, rank - 1)
            else:
                protector = chess.square(file + dx, rank + 1)

            protector_piece = board.piece_at(protector)
            if protector_piece and protector_piece.piece_type == chess.PAWN and protector_piece.color == pawn_color:
                return True
    return False

def evaluate_pawn_structure(board, square):
    """Evaluate pawn structure (isolated, passed, etc.)"""
    score = 0
    piece = board.piece_at(square)
    if not piece or piece.piece_type != chess.PAWN:
        return score

    file = chess.square_file(square)
    adjacent_files = [f for f in [file-1, file+1] if 0 <= f < 8]

    # Check for isolated pawns
    is_isolated = True
    for f in adjacent_files:
        if any(p.piece_type == chess.PAWN and p.color == piece.color
              for _, p in board.piece_map().items()
              if chess.square_file(_) == f):
            is_isolated = False
            break
    if is_isolated:
        score -= 0.3

    # Check for passed pawns
    is_passed = True
    rank = chess.square_rank(square)
    for f in [file-1, file, file+1]:
        if not (0 <= f < 8):
            continue
        for r in range(8):
            if (piece.color == chess.WHITE and r < rank) or (piece.color == chess.BLACK and r > rank):
                continue
            s = chess.square(f, r)
            p = board.piece_at(s)
            if p and p.piece_type == chess.PAWN and p.color != piece.color:
                is_passed = False
                break
        if not is_passed:
            break
    if is_passed:
        score += 0.5

    return score

def evaluate_king_safety(board, square):
    """Evaluate king safety (castle status, pawn shield, etc.)"""
    score = 0
    piece = board.piece_at(square)
    if not piece or piece.piece_type != chess.KING:
        return score

    # Bonus for castling
    if board.has_kingside_castling_rights(piece.color):
        score += 0.2
    if board.has_queenside_castling_rights(piece.color):
        score += 0.2

    # Evaluate pawn shield
    file = chess.square_file(square)
    rank = chess.square_rank(square)
    pawn_shield = 0

    for f in [file-1, file, file+1]:
        if not (0 <= f < 8):
            continue
        if piece.color == chess.WHITE:
            shield_rank = rank - 1
        else:
            shield_rank = rank + 1

        if 0 <= shield_rank < 8:
            s = chess.square(f, shield_rank)
            p = board.piece_at(s)
            if p and p.piece_type == chess.PAWN and p.color == piece.color:
                pawn_shield += 1

    score += pawn_shield * 0.1

    # Penalty when in check
    if board.is_check():
        score -= 0.5

    return score

def evaluate_tactics(board, piece_values):
    """Evaluate tactical elements (pins, forks, etc.)"""
    score = 0

    # Check for pins
    for square in chess.SQUARES:
        if board.is_pinned(board.turn, square):
            score -= 0.3

    # Check for potential forks
    for move in board.legal_moves:
        board.push(move)
        if board.is_check() and len(list(board.legal_moves)) == 0:
            board.pop()
            score += 1  # Checkmate would be even better than fork
            break

        attackers = board.attackers(board.turn, move.to_square)
        if len(attackers) > 1:
            # Potential fork detected
            score -= 0.5
        board.pop()

    # Check for hanging pieces
    for square, piece in board.piece_map().items():
        if piece.color == board.turn:
            attackers = board.attackers(not board.turn, square)
            if attackers:
                defenders = board.attackers(board.turn, square)
                if len(attackers) > len(defenders):
                    score -= piece_values.get(piece.piece_type, 0) * 0.5

    return score

def minimax(board, depth, alpha, beta, is_maximizing_player):
    if depth == 0 or board.is_game_over():
        return evaluate_board(board)

    if is_maximizing_player:
        max_eval = -float('inf')
        for move in board.legal_moves:
            board.push(move)
            eval = minimax(board, depth - 1, alpha, beta, False)
            board.pop()
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float('inf')
        for move in board.legal_moves:
            board.push(move)
            eval = minimax(board, depth - 1, alpha, beta, True)
            board.pop()
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

def find_best_move(board, depth):
    best_move = None
    best_eval = -float('inf') if board.turn == chess.WHITE else float('inf')
    alpha = -float('inf')
    beta = float('inf')
    is_maximizing_player = board.turn == chess.WHITE

    for move in board.legal_moves:
        board.push(move)
        eval = minimax(board, depth - 1, alpha, beta, not is_maximizing_player)
        board.pop()

        if is_maximizing_player:
            if eval > best_eval:
                best_eval = eval
                best_move = move
            alpha = max(alpha, eval)
        else:
            if eval < best_eval:
                best_eval = eval
                best_move = move
            beta = min(beta, eval)

    return best_move

def play_game_cli(bot_depth=3):
    """Main game loop for CLI interface"""
    while True:
        board = chess.Board()
        print("Welcome to the Chess Bot!")

        # Player color selection
        while True:
            user_color_input = input("Choose your color (white or black): ").lower()
            if user_color_input == 'white':
                user_is_white = True
                print("You are playing as White.")
                break
            elif user_color_input == 'black':
                user_is_white = False
                print("You are playing as Black.")
                break
            else:
                print("Invalid input. Please type 'white' or 'black'.")

        print("Type 'reset' to start a new game or 'quit' to exit.")
        print("="*50)

        while not board.is_game_over():
            # Display board
            print("\nCurrent board:")
            print(board)
            print("="*50)

            if (board.turn == chess.WHITE and user_is_white) or (board.turn == chess.BLACK and not user_is_white):
                # Player's turn
                while True:
                    user_input = input("Your move (e.g., e2e4, Nf3) or 'reset/quit': ").strip()
                    if user_input.lower() == 'reset':
                        print("Restarting game...")
                        break
                    if user_input.lower() == 'quit':
                        return

                    try:
                        move = board.parse_uci(user_input) if len(user_input) == 4 else board.parse_san(user_input)
                        if move in board.legal_moves:
                            board.push(move)
                            break
                        else:
                            print("Illegal move. Try again.")
                    except ValueError:
                        print("Invalid move format. Try again or type 'reset/quit'.")

                if user_input.lower() == 'reset':
                    break
            else:
                # Bot's turn
                print("Bot is thinking...")
                bot_move = find_best_move(board, bot_depth)
                if bot_move:
                    print(f"Bot plays: {bot_move.uci()}")
                    board.push(bot_move)
                else:
                    print("Bot has no legal moves.")
                    break

            # Check game status
            if board.is_checkmate():
                winner = "Black" if board.turn == chess.WHITE else "White"
                print(f"\nCHECKMATE! {winner} wins!")
            elif board.is_stalemate():
                print("\nSTALEMATE! Game is drawn.")
            elif board.is_insufficient_material():
                print("\nDraw by insufficient material.")
            elif board.is_fifty_moves():
                print("\nDraw by fifty-move rule.")
            elif board.is_repetition():
                print("\nDraw by repetition.")

        # Post-game options
        if board.is_game_over():
            print("\nFinal board:")
            print(board)
            print("Game result:", board.result())

            play_again = input("Play again? (y/n): ").lower()
            if play_again != 'y':
                break

if __name__ == "__main__":
    play_game_cli(bot_depth=3)


Welcome to the Chess Bot!
